## Tutorial of changing how TinyStories end with simple interventions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/frankaging/align-transformers/blob/main/tutorials/Change%20how%20TinyStories%20end%20with%20interventions.ipynb)

In [1]:
__author__ = "Zhengxuan Wu"
__version__ = "10/08/2023"

### Overview

Most of the tutorials focus on a single token generation task (e.g., capital change, price tagging task). Most of the real-world tasks are multi-token generations (e.g., ChatGPT). In this tutorial, we show how to intervene on a generic language generation task.

In [2]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import transformers
    import sys
    sys.path.append("align-transformers/")
except ModuleNotFoundError:
    !git clone https://github.com/frankaging/align-transformers.git
    !pip install -r align-transformers/requirements.txt
    import sys
    sys.path.append("align-transformers/")

In [3]:
import sys
sys.path.append("..")

import torch
import pandas as pd
from models.utils import print_forward_hooks, remove_forward_hooks
from models.configuration_alignable_model import AlignableRepresentationConfig, AlignableConfig
from models.alignable_base import AlignableModel
from models.interventions import AdditionIntervention, SubstractionIntervention
from models.gpt_neo.modelings_alignable_gpt_neo import create_gpt_neo

%config InlineBackend.figure_formats = ['svg']

[2023-12-14 17:52:15,227] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


### Original generation with TinyStories-33M

In [4]:
config, tokenizer, tinystory = create_gpt_neo()

loaded model


In [5]:
prompt = "Once upon a time there was"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = tinystory.generate(input_ids, max_length = 512, num_beams=1)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time there was a little girl named Lucy. She was three years old and loved to explore. One day, Lucy was walking in the park when she saw a big, red balloon. She was so excited and ran over to it.

"Can I have it?" she asked.

"No," said her mom. "It's too big for you. You can't have it."

Lucy was sad, but then she saw a small, red balloon. She smiled and said, "I want that one!"

Her mom smiled and said, "Okay, let's go get it."

So they went to the balloon and Lucy was so happy. She held the balloon tight and ran around the park with it. She laughed and smiled and had so much fun.

When it was time to go home, Lucy hugged the balloon and said, "I love you, balloon!"

Her mom smiled and said, "I love you too, Lucy."



### Set-up interventions on language generation

In [6]:
def activation_addition_position_config(
    model_type, intervention_type, 
    start_layer_idx, end_layer_idx
):
    alignable_config = AlignableConfig(
        alignable_model_type=model_type,
        alignable_representations=[
            AlignableRepresentationConfig(
                i,                 # layer
                intervention_type, # intervention type
                "pos",             # intervention unit
                1                  # max number of unit
            ) for i in range(start_layer_idx, end_layer_idx)
        ],
        alignable_interventions_type=AdditionIntervention,
    )
    return alignable_config

config, tokenizer, tinystory = create_gpt_neo()
alignable_config = activation_addition_position_config(
    type(tinystory), "mlp_output", 0, 4
)
alignable = AlignableModel(alignable_config, tinystory)

loaded model


In [7]:
sad_token_id = tokenizer(" Sad")["input_ids"][0]
happy_token_id = tokenizer(" Happy")["input_ids"][0]

beta = 0.3 # very hacky way to control the effect
sad_embedding = tinystory.transformer.wte(
    torch.tensor(sad_token_id)).clone().unsqueeze(0).unsqueeze(0) # make it a fake batch
happy_embedding = tinystory.transformer.wte(
    torch.tensor(happy_token_id)).clone().unsqueeze(0).unsqueeze(0) # make it a fake batch
sad_embedding *= beta
happy_embedding *= beta

activations_sad_sources = dict(
    zip(alignable.sorted_alignable_keys, 
        [sad_embedding]*len(alignable.sorted_alignable_keys))
)
activations_happy_sources = dict(
    zip(alignable.sorted_alignable_keys, 
        [happy_embedding]*len(alignable.sorted_alignable_keys))
)
# we intervene on all of the mlp output
alignable.sorted_alignable_keys

['layer.0.repr.mlp_output.unit.pos.nunit.1#0',
 'layer.1.repr.mlp_output.unit.pos.nunit.1#0',
 'layer.2.repr.mlp_output.unit.pos.nunit.1#0',
 'layer.3.repr.mlp_output.unit.pos.nunit.1#0']

### Adding a little bit of "sadness" into the story

In [8]:
base = "Once upon a time there was"

inputs = tokenizer(base, return_tensors="pt")
base_outputs, counterfactual_outputs = alignable.generate(
    inputs, 
    unit_locations={
        "sources->base": (
            [[[0]]]*tinystory.config.num_layers, # a single token embeddings
            [[[0]]]*tinystory.config.num_layers  # the last token of the prompt
        )
    },
    activations_sources=activations_sad_sources,
    intervene_on_prompt=False,
    max_length=256, num_beams=1,
)
counterfactual_text = tokenizer.decode(
    counterfactual_outputs[0], skip_special_tokens=True)
print(counterfactual_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time there was a little girl named Lucy. She was three years old and loved to explore. One day, Lucy was walking in the park when she saw a big, red balloon. She was so excited and ran over to it. She reached out to grab it, but it was too high.

Suddenly, a kind old man appeared. He said, "I can help you get the balloon, but first you must do something for me." Lucy was confused, but she agreed. The old man said, "I need you to help me pull the balloon down from the tree."

Lucy was scared, but she was brave and she did it. The old man grabbed the balloon and handed it to Lucy. She was so happy and thanked the old man.

The old man said, "You are very brave and strong. I'm glad I could help you." Lucy smiled and said, "Thank you!" She hugged the balloon tightly and ran off to show her mom her new prize.



### Adding a little bit of "happiness" into the story

In [9]:
base = "Once upon a time there was"

inputs = tokenizer(base, return_tensors="pt")
base_outputs, counterfactual_outputs = alignable.generate(
    inputs, 
    unit_locations={
        "sources->base": (
            [[[0]]]*tinystory.config.num_layers, # a single token embeddings
            [[[0]]]*tinystory.config.num_layers  # the last token of the prompt
        )
    },
    activations_sources=activations_happy_sources,
    intervene_on_prompt=False,
    max_length=256, num_beams=1,
)
counterfactual_text = tokenizer.decode(
    counterfactual_outputs[0], skip_special_tokens=True)
print(counterfactual_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time there was a little girl named Lucy. She was three years old and loved to explore. One day, Lucy was walking in the park when she saw a big, red balloon. She was so excited and wanted to play with it.

But then, a big, mean man came and said, "That balloon is mine! You can't have it!" Lucy was very sad and started to cry.

The man said, "I'm sorry, but I need the balloon for my work. You can have it if you want."

Lucy was so happy and said, "Yes please!" She took the balloon and ran away.

But then, the man said, "Wait! I have an idea. Let's make a deal. If you can guess what I'm going to give you, then you can have the balloon."

Lucy thought for a moment and then said, "I guess I'll have to get the balloon."

The man smiled and said, "That's a good guess! Here you go."

Lucy was so happy and thanked the man. She hugged the balloon and ran off to show her mom.

The end.

